<a href="https://colab.research.google.com/github/0v3r-9000/trc204.github.io/blob/main/Multiple_Regression_Math_Score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio

# Import necessary libraries
from sklearn import preprocessing
import gradio as gr
import pandas as pd
from sklearn import linear_model
from sklearn.impute import SimpleImputer
import numpy as np

# Install gradio (only needs to be done once per session)
# %%
!pip install gradio

# Load the dataset
try:
    df = pd.read_csv("StudentsPerformance.csv")
except FileNotFoundError:
    !mkdir -p sample_data
    # No longer downloading Depression.csv
    # !wget https://raw.githubusercontent.com/daniel-dc-cd/data_science/main/module_4_ML/data/Depression.csv -P sample_data
    # Instead, create a dummy StudentsPerformance.csv if it's not found
    df = pd.DataFrame(columns=['gender', 'race/ethnicity', 'parental level of education', 'test preparation course', 'reading score', 'writing score', 'math score'])
    df.to_csv("StudentsPerformance.csv", index=False)
    df = pd.read_csv("StudentsPerformance.csv")

# Select the specified columns
df = df[['gender', 'race/ethnicity', 'parental level of education', 'test preparation course', 'reading score', 'writing score', 'math score']]

# Identify string columns
string_columns = df.select_dtypes(include=['object']).columns

# Initialize LabelEncoder
label_encoder = preprocessing.LabelEncoder()

# Apply label encoding to each string column
for col in string_columns:
    df[col] = label_encoder.fit_transform(df[col])

# Check for and handle missing values
numeric_cols = df.select_dtypes(include=np.number).columns
imputer = SimpleImputer(strategy='mean')
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

# Create X and y
X = df[['gender', 'race/ethnicity', 'parental level of education', 'test preparation course', 'reading score', 'writing score']]
y = df['math score']

# Initialize and train the model
regr = linear_model.LinearRegression()
regr.fit(X, y)

def predict_math(gender, race_ethnicity, parent_education, test_preparation, reading_score, writing_score):
    """
    Predicts the math score based on input features.
    """
    prediction_input = np.array([[gender, race_ethnicity, parent_education, test_preparation, reading_score, writing_score]])
    predicted_math_score = regr.predict(prediction_input)
    return predicted_math_score[0]

def generate_math_recommendations(gender, race_ethnicity, parent_education, test_preparation, reading_score, writing_score, predicted_math_score):
    """
    Generates recommendations based on the input values and prediction.
    """
    recommendations = []

    if predicted_math_score < 70:  # If the model predicts a low score
        recommendations.append("It seems like your math score might be low.")
        recommendations.append("Here are some suggestions that might help:")

        if reading_score < 70:
            recommendations.append("- Try to read more books.")
        if writing_score < 70:
            recommendations.append("- Try to practice writing different texts.")
        if test_preparation == 0:
            recommendations.append("- Consider taking a test preparation course.")
        recommendations.append("- Practice solving math problems.")
        recommendations.append("Remember that I am just an AI, you should always consult with a professional if you need help.")

    else:
        recommendations.append("The model suggests you are doing well with your math.")
        recommendations.append("Here are some suggestions to continue improving:")

        if reading_score > 70:
            recommendations.append("- Continue reading to improve your knowledge.")
        if writing_score > 70:
            recommendations.append("- Keep practicing your writing to better solve the problems.")
        if test_preparation == 1:
            recommendations.append("- Keep with the test preparation course.")
        recommendations.append("- Practice solving math problems.")

        recommendations.append("You are doing a great job, keep it up!")

    return "\n".join(recommendations)

def chatbot_response(user_message, history, input_values):
    """
    Handles the chatbot's responses and logic.
    """
    history = history or []
    response = ""
    stage = input_values.get("stage", 0)

    if stage == 0:
        response = "Hello! I'm here to help you assess your math score. Let's start with some questions. What is your gender? (0 for female, 1 for male)"
        input_values["stage"] = 1
    elif stage == 1:
        try:
            input_values["gender"] = int(user_message)
            response = "What is your race/ethnicity? (0-4)"
            input_values["stage"] = 2
        except ValueError:
            response = "Please enter a valid number for your gender."
    elif stage == 2:
        try:
            input_values["race_ethnicity"] = int(user_message)
            response = "What is your parent's level of education? (0-5)"
            input_values["stage"] = 3
        except ValueError:
            response = "Please enter a valid number for race/ethnicity (0-4)."
    elif stage == 3:
        try:
            input_values["parent_education"] = int(user_message)
            response = "Did you take a test preparation course? (0 for no, 1 for yes)"
            input_values["stage"] = 4
        except ValueError:
            response = "Please enter a valid number for parent's level of education. (0-5)"
    elif stage == 4:
        try:
            input_values["test_preparation"] = int(user_message)
            response = "What is your reading score?"
            input_values["stage"] = 5
        except ValueError:
            response = "Please enter a valid number for the test preparation course. (0, 1)"
    elif stage == 5:
        try:
            input_values["reading_score"] = int(user_message)
            response = "What is your writing score?"
            input_values["stage"] = 6
        except ValueError:
            response = "Please enter a valid number for the reading score."
    elif stage == 6:
        try:
            input_values["writing_score"] = int(user_message)
            input_values["stage"] = 7
            predicted_math_score = predict_math(
                input_values["gender"],
                input_values["race_ethnicity"],
                input_values["parent_education"],
                input_values["test_preparation"],
                input_values["reading_score"],
                input_values["writing_score"]
            )
            recommendations = generate_math_recommendations(
                input_values["gender"],
                input_values["race_ethnicity"],
                input_values["parent_education"],
                input_values["test_preparation"],
                input_values["reading_score"],
                input_values["writing_score"],
                predicted_math_score
            )
            response = f"Based on the information you've provided, your predicted math score is: {predicted_math_score:.4f}\n\nRecommendations:\n{recommendations}"
        except ValueError:
            response = "Please enter a valid number for the writing score."
    elif stage == 7:
        response = "I've given you the recommendations for now. Please, if you need it, you can always consult with a professional. If you want to restart, type reset"
        if user_message.lower() == "reset":
          input_values = {"stage": 0}
          response = "Hello! I'm here to help you assess your math score. Let's start with some questions. What is your gender? (0 for female, 1 for male)"
          input_values["stage"] = 1

    history.append((user_message, response))
    return history, input_values

with gr.Blocks(theme=gr.themes.Soft(), title="Math Score Panda") as demo:
    # Title for the entire app
    gr.Markdown("# Math Score Panda")

    # Description for the entire app
    gr.Markdown("This app predicts your math score and offers support through a conversational AI.")

    # Create Tabs
    with gr.Tab("Therapist AI"):
        # Chatbot components
        chatbot = gr.Chatbot(label="Therapist AI")
        user_message = gr.Textbox(label="Your Message", placeholder="Type your message here...")
        send_button = gr.Button("Send")
        input_values = gr.State({"stage": 0})

        # Send button click to the chatbot
        send_button.click(
            fn=chatbot_response,
            inputs=[user_message, chatbot, input_values],
            outputs=[chatbot, input_values]
        )
        user_message.submit(
            fn=chatbot_response,
            inputs=[user_message, chatbot, input_values],
            outputs=[chatbot, input_values]
        )

demo.launch(share=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 1.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


KeyboardInterrupt: 

In [ ]:


# Import necessary libraries
from sklearn import preprocessing
import gradio as gr
import pandas as pd
from sklearn import linear_model
from sklearn.impute import SimpleImputer
import numpy as np



# Load the dataset
try:
    df = pd.read_csv("StudentsPerformance.csv")
except FileNotFoundError:
    !mkdir -p sample_data
    # No longer downloading Depression.csv
    # !wget https://raw.githubusercontent.com/daniel-dc-cd/data_science/main/module_4_ML/data/Depression.csv -P sample_data
    # Instead, create a dummy StudentsPerformance.csv if it's not found
    df = pd.DataFrame(columns=['gender', 'race/ethnicity', 'parental level of education', 'test preparation course', 'reading score', 'writing score', 'math score'])
    df.to_csv("StudentsPerformance.csv", index=False)
    df = pd.read_csv("StudentsPerformance.csv")

# Select the specified columns
df = df[['gender', 'race/ethnicity', 'parental level of education', 'test preparation course', 'reading score', 'writing score', 'math score']]

# Identify string columns
string_columns = df.select_dtypes(include=['object']).columns

# Initialize LabelEncoder
label_encoder = preprocessing.LabelEncoder()

# Apply label encoding to each string column
for col in string_columns:
    df[col] = label_encoder.fit_transform(df[col])

# Check for and handle missing values
numeric_cols = df.select_dtypes(include=np.number).columns
imputer = SimpleImputer(strategy='mean')
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

# Create X and y
X = df[['gender', 'race/ethnicity', 'parental level of education', 'test preparation course', 'reading score', 'writing score']]
y = df['math score']

# Initialize and train the model
regr = linear_model.LinearRegression()
regr.fit(X, y)

def predict_math(gender, race_ethnicity, parent_education, test_preparation, reading_score, writing_score):
    """
    Predicts the math score based on input features.
    """
    prediction_input = np.array([[gender, race_ethnicity, parent_education, test_preparation, reading_score, writing_score]])
    predicted_math_score = regr.predict(prediction_input)
    return predicted_math_score[0]

def generate_math_recommendations(gender, race_ethnicity, parent_education, test_preparation, reading_score, writing_score, predicted_math_score):
    """
    Generates recommendations based on the input values and prediction.
    """
    recommendations = []

    if predicted_math_score < 70:  # If the model predicts a low score
        recommendations.append("It seems like your math score might be low.")
        recommendations.append("Here are some suggestions that might help:")

        if reading_score < 70:
            recommendations.append("- Try to read more books.")
        if writing_score < 70:
            recommendations.append("- Try to practice writing different texts.")
        if test_preparation == 0:
            recommendations.append("- Consider taking a test preparation course.")
        recommendations.append("- Practice solving math problems.")
        recommendations.append("Remember that I am just an AI, you should always consult with a professional if you need help.")

    else:
        recommendations.append("The model suggests you are doing well with your math.")
        recommendations.append("Here are some suggestions to continue improving:")

        if reading_score > 70:
            recommendations.append("- Continue reading to improve your knowledge.")
        if writing_score > 70:
            recommendations.append("- Keep practicing your writing to better solve the problems.")
        if test_preparation == 1:
            recommendations.append("- Keep with the test preparation course.")
        recommendations.append("- Practice solving math problems.")

        recommendations.append("You are doing a great job, keep it up!")

    return "\n".join(recommendations)

def chatbot_response(user_message, history, input_values):
    """
    Handles the chatbot's responses and logic.
    """
    history = history or []
    response = ""
    stage = input_values.get("stage", 0)

    if stage == 0:
        response = "Hello! I'm here to help you assess your math score. Let's start with some questions. What is your gender? (0 for female, 1 for male)"
        input_values["stage"] = 1
    elif stage == 1:
        try:
            input_values["gender"] = int(user_message)
            response = "What is your race/ethnicity? (0-4)"
            input_values["stage"] = 2
        except ValueError:
            response = "Please enter a valid number for your gender."
    elif stage == 2:
        try:
            input_values["race_ethnicity"] = int(user_message)
            response = "What is your parent's level of education? (0-5)"
            input_values["stage"] = 3
        except ValueError:
            response = "Please enter a valid number for race/ethnicity (0-4)."
    elif stage == 3:
        try:
            input_values["parent_education"] = int(user_message)
            response = "Did you take a test preparation course? (0 for no, 1 for yes)"
            input_values["stage"] = 4
        except ValueError:
            response = "Please enter a valid number for parent's level of education. (0-5)"
    elif stage == 4:
        try:
            input_values["test_preparation"] = int(user_message)
            response = "What is your reading score?"
            input_values["stage"] = 5
        except ValueError:
            response = "Please enter a valid number for the test preparation course. (0, 1)"
    elif stage == 5:
        try:
            input_values["reading_score"] = int(user_message)
            response = "What is your writing score?"
            input_values["stage"] = 6
        except ValueError:
            response = "Please enter a valid number for the reading score."
    elif stage == 6:
        try:
            input_values["writing_score"] = int(user_message)
            input_values["stage"] = 7
            predicted_math_score = predict_math(
                input_values["gender"],
                input_values["race_ethnicity"],
                input_values["parent_education"],
                input_values["test_preparation"],
                input_values["reading_score"],
                input_values["writing_score"]
            )
            recommendations = generate_math_recommendations(
                input_values["gender"],
                input_values["race_ethnicity"],
                input_values["parent_education"],
                input_values["test_preparation"],
                input_values["reading_score"],
                input_values["writing_score"],
                predicted_math_score
            )
            response = f"Based on the information you've provided, your predicted math score is: {predicted_math_score:.4f}\n\nRecommendations:\n{recommendations}"
        except ValueError:
            response = "Please enter a valid number for the writing score."
    elif stage == 7:
        response = "I've given you the recommendations for now. Please, if you need it, you can always consult with a professional. If you want to restart, type reset"
        if user_message.lower() == "reset":
          input_values = {"stage": 0}
          response = "Hello! I'm here to help you assess your math score. Let's start with some questions. What is your gender? (0 for female, 1 for male)"
          input_values["stage"] = 1

    history.append((user_message, response))
    return history, input_values

with gr.Blocks(theme=gr.themes.Soft(), title="Math Score Panda") as demo:
    # Title for the entire app
    gr.Markdown("# Math Score Panda")

    # Description for the entire app
    gr.Markdown("This app predicts your math score and offers support through a conversational AI.")

    # Create Tabs
    with gr.Tab("Therapist AI"):
        # Chatbot components
        chatbot = gr.Chatbot(label="Therapist AI")
        user_message = gr.Textbox(label="Your Message", placeholder="Type your message here...")
        send_button = gr.Button("Send")
        input_values = gr.State({"stage": 0})

        # Send button click to the chatbot
        send_button.click(
            fn=chatbot_response,
            inputs=[user_message, chatbot, input_values],
            outputs=[chatbot, input_values]
        )
        user_message.submit(
            fn=chatbot_response,
            inputs=[user_message, chatbot, input_values],
            outputs=[chatbot, input_values]
        )

demo.launch(share=True)

ValueError: Found array with 0 sample(s) (shape=(0, 7)) while a minimum of 1 is required by SimpleImputer.

In [8]:
import gradio as gr
import pandas as pd
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.impute import SimpleImputer
import numpy as np

# Load and preprocess the dataset
try:
    df = pd.read_csv("StudentsPerformance.csv")
except FileNotFoundError:
    # Create a dummy dataset if the file is not found
    df = pd.DataFrame(columns=['gender', 'race/ethnicity', 'parental level of education',
                               'test preparation course', 'reading score',
                               'writing score', 'math score'])
    df.to_csv("StudentsPerformance.csv", index=False)
    df = pd.read_csv("StudentsPerformance.csv")

# Select relevant columns
df = df[['gender', 'race/ethnicity', 'parental level of education',
         'test preparation course', 'reading score', 'writing score', 'math score']]

# Encode categorical columns
string_columns = df.select_dtypes(include=['object']).columns
label_encoder = preprocessing.LabelEncoder()
for col in string_columns:
    df[col] = label_encoder.fit_transform(df[col])

# Handle missing values
numeric_cols = df.select_dtypes(include=np.number).columns
imputer = SimpleImputer(strategy='mean')
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

# Prepare features (X) and target (y)
X = df[['gender', 'race/ethnicity', 'parental level of education',
        'test preparation course', 'reading score', 'writing score']]
y = df['math score']

print(df.shape)
print(df.head())


# Train the linear regression model
regr = linear_model.LinearRegression()
regr.fit(X, y)

# Define the prediction and recommendation functions
def predict_and_recommend(gender, race_ethnicity, parent_education, test_preparation, reading_score, writing_score):
    """
    Predict the math score and provide recommendations based on input features.
    """
    # Predict math score
    input_data = np.array([[gender, race_ethnicity, parent_education, test_preparation,
                            reading_score, writing_score]])
    predicted_math_score = regr.predict(input_data)[0]

    # Generate recommendations
    recommendations = []
    if predicted_math_score < 70:
        recommendations.append("It seems like your math score might be low.")
        recommendations.append("Here are some suggestions that might help:")
        if reading_score < 70:
            recommendations.append("- Try to read more books.")
        if writing_score < 70:
            recommendations.append("- Practice writing different types of texts.")
        if test_preparation == 0:
            recommendations.append("- Consider taking a test preparation course.")
        recommendations.append("- Practice solving math problems.")
    else:
        recommendations.append("You're doing well in math! Keep it up!")
        recommendations.append("Here are some tips to maintain or improve further:")
        if reading_score > 70:
            recommendations.append("- Continue reading to expand your knowledge.")
        if writing_score > 70:
            recommendations.append("- Keep practicing your writing skills.")
        if test_preparation == 1:
            recommendations.append("- Continue with the test preparation course.")
        recommendations.append("- Keep practicing math problems regularly.")

    return f"Predicted Math Score: {predicted_math_score:.2f}\n\nRecommendations:\n" + "\n".join(recommendations)

# Gradio UI
with gr.Blocks(theme=gr.themes.Soft(), title="Math Score Predictor") as demo:
    gr.Markdown("# Math Score Predictor")
    gr.Markdown("This app predicts your math score and provides personalized recommendations.")

    # Input fields for the user
    gender = gr.Radio(label="Gender", choices=[0, 1], type="index",
                      value=0, info="0 = Female, 1 = Male")
    race_ethnicity = gr.Dropdown(label="Race/Ethnicity", choices=[0, 1, 2, 3, 4],
                                 type="index", value=0, info="Select your race/ethnicity (0-4)")
    parent_education = gr.Dropdown(label="Parental Level of Education",
                                   choices=[0, 1, 2, 3, 4, 5], type="index", value=0)
    test_preparation = gr.Radio(label="Test Preparation Course",
                                choices=[0, 1], type="index", value=0, info="0 = No, 1 = Yes")
    reading_score = gr.Slider(label="Reading Score", minimum=0, maximum=100, step=1, value=50)
    writing_score = gr.Slider(label="Writing Score", minimum=0, maximum=100, step=1, value=50)

    # Output field for results
    result = gr.Textbox(label="Results", lines=10)

    # Button to trigger prediction
    predict_button = gr.Button("Predict and Get Recommendations")
    predict_button.click(predict_and_recommend,
                         inputs=[gender, race_ethnicity, parent_education,
                                 test_preparation, reading_score, writing_score],
                         outputs=result)

# Launch the app
demo.launch(share=True)


(1000, 7)
   gender  race/ethnicity  parental level of education  \
0     0.0             1.0                          1.0   
1     0.0             2.0                          4.0   
2     0.0             1.0                          3.0   
3     1.0             0.0                          0.0   
4     1.0             2.0                          4.0   

   test preparation course  reading score  writing score  math score  
0                      1.0           72.0           74.0        72.0  
1                      0.0           90.0           88.0        69.0  
2                      1.0           95.0           93.0        90.0  
3                      1.0           57.0           44.0        47.0  
4                      1.0           78.0           75.0        76.0  
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://da2b6d2e0527b4bf12.gradio.live

This share link expires in 72 hours. For free permanent hosting 

In [2]:
import pandas as pd

# Attempt to load the dataset
try:
    df = pd.read_csv("StudentsPerformance.csv")
    print("Dataset loaded successfully.")
    print(df.head())  # Check the first few rows
except FileNotFoundError:
    print("File not found. Ensure 'StudentsPerformance.csv' exists in the directory.")


Dataset loaded successfully.
   gender race/ethnicity parental level of education         lunch  \
0  female        group B           bachelor's degree      standard   
1  female        group C                some college      standard   
2  female        group B             master's degree      standard   
3    male        group A          associate's degree  free/reduced   
4    male        group C                some college      standard   

  test preparation course  math score  reading score  writing score  
0                    none          72             72             74  
1               completed          69             90             88  
2                    none          90             95             93  
3                    none          47             57             44  
4                    none          76             78             75  


In [3]:
# Select relevant columns
required_columns = ['gender', 'race/ethnicity', 'parental level of education',
                    'test preparation course', 'reading score', 'writing score', 'math score']

if all(col in df.columns for col in required_columns):
    df = df[required_columns]
    print("Columns selected successfully.")
else:
    missing = [col for col in required_columns if col not in df.columns]
    print(f"Missing columns: {missing}")


Columns selected successfully.


In [4]:
from sklearn import preprocessing

# Identify string columns
string_columns = df.select_dtypes(include=['object']).columns
print(f"String columns identified: {string_columns}")

# Apply label encoding
label_encoder = preprocessing.LabelEncoder()
for col in string_columns:
    try:
        df[col] = label_encoder.fit_transform(df[col])
        print(f"Encoded column: {col}")
    except Exception as e:
        print(f"Error encoding column {col}: {e}")


String columns identified: Index(['gender', 'race/ethnicity', 'parental level of education',
       'test preparation course'],
      dtype='object')
Encoded column: gender
Encoded column: race/ethnicity
Encoded column: parental level of education
Encoded column: test preparation course


In [5]:
import numpy as np
from sklearn.impute import SimpleImputer

# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())

# Handle missing values in numeric columns
numeric_cols = df.select_dtypes(include=np.number).columns
print(f"Numeric columns identified: {numeric_cols}")

print("Shape of numeric columns DataFrame before imputation:")
print(df[numeric_cols].shape)
print(df[numeric_cols].head())

print("Rows with missing values in numeric columns:")
print(df[numeric_cols].isnull().any(axis=1))



try:
    imputer = SimpleImputer(strategy='mean')
    df[numeric_cols] = imputer.fit_transform(df[numeric_cols])
    print("Imputation applied successfully.")
except Exception as e:
    print(f"Error during imputation: {e}")


Missing values per column:
gender                         0
race/ethnicity                 0
parental level of education    0
test preparation course        0
reading score                  0
writing score                  0
math score                     0
dtype: int64
Numeric columns identified: Index(['gender', 'race/ethnicity', 'parental level of education',
       'test preparation course', 'reading score', 'writing score',
       'math score'],
      dtype='object')
Shape of numeric columns DataFrame before imputation:
(1000, 7)
   gender  race/ethnicity  parental level of education  \
0       0               1                            1   
1       0               2                            4   
2       0               1                            3   
3       1               0                            0   
4       1               2                            4   

   test preparation course  reading score  writing score  math score  
0                        1             

In [6]:
from sklearn import linear_model

# Define features (X) and target (y)
X = df[['gender', 'race/ethnicity', 'parental level of education',
        'test preparation course', 'reading score', 'writing score']]
y = df['math score']

# Train the model
try:
    regr = linear_model.LinearRegression()
    regr.fit(X, y)
    print("Model trained successfully.")
except Exception as e:
    print(f"Error during model training: {e}")


Model trained successfully.


In [7]:
# Define feature names
feature_names = ['gender', 'race/ethnicity', 'parental level of education',
                 'test preparation course', 'reading score', 'writing score']

# Test the prediction with proper feature names
try:
    sample_input = pd.DataFrame([[0, 1, 2, 1, 80, 75]], columns=feature_names)
    predicted_score = regr.predict(sample_input)
    print(f"Predicted Math Score: {predicted_score[0]:.2f}")
except Exception as e:
    print(f"Error during prediction: {e}")


Predicted Math Score: 67.84
